# Import Required Libraries

In [1]:
import re
import unicodedata
from collections import defaultdict
import networkx as nx
from itertools import combinations
import difflib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import jdatetime


### Load Dataset

In [2]:
df = pd.read_csv('reviews.csv')

### Basic Cleaning

In [3]:
"""
During EDA, we determined that nearly all missing values were due to: 
survey images, professor suggestion polls, or various single messages 
(like pinned posts, guides, or initial channel messages).
We may revisit EDA-related code later to extract any potentially useful information from these if possible.
For now, we removing them as we did during the EDA phase.
"""

# Just 2 Special Cases
df.loc[df['message_id'] == 816, ['department', 'comment_text', 'parse_error']] = [
    'معارف', 
    'خییلللللی استاد محشرین از دسشون ندین ۴ نمره اضافی دارن برای نهاد که جواباش پیدا میشه😁 امتحاناشونم میانترم کپی ترم پیش و پایانترم از یه فایل ۲۰ صفحه ای بود',
    None
]

df.loc[df['message_id'] == 252, 'comment_text'] = np.nan

df = df.dropna(subset=['term'])

In [4]:
print("Total reviews:", len(df))
print("Unique professors:", df["professor_name_raw"].nunique())
print("Departments:", df["department"].nunique())

Total reviews: 4276
Unique professors: 1296
Departments: 18


In [5]:
df['comment_text'].isna().sum()

np.int64(16)

### Term Normalization

In [6]:
temp_grouped = {
 ('قبل از سال 94',): 'before_94',

 ('بهمن 94',): 'second_1394',

 ('تابستان 95',): 'summer_1395',
 
 ('مهر 96',): 'first_1396',
 ('تابستان 96',): 'summer_1396',
 
 ('مهر 97',): 'first_1397',
 ('بهمن 97',): 'second_1397',
 
 ('مهر 98',): 'first_1398',
 ('بهمن 98',): 'second_1398',
 
 ('مهر 99',): 'first_1399',
 ('بهمن 99', 'بهمن ۹۹'): 'second_1399',
 ('تابستان 99',): 'summer_1399',
 
 ('مهر 00', 'مهر 1400', 'مهر ۰۰', 'مهر ..', 'مهر ١٤٠٠', 'مهر ۱۴۰۰', 'مهر۱۴۰۰', 'پاییز ۱۴۰۰'): 'first_1400',
 ('4002', 'بهمن 1400', 'بهمن ۱۴۰۰'): 'second_1400',
 ('تابستان 1400', 'تابستان۱۴۰۰'): 'summer_1400',
 ('بهمن 99 ،مهر 1400', 'مهر و بهمن 1400', 'مهر و بهمن ۱۴۰۰', 'هم ترم مهر ۱۴۰۰ و ترم بهمن ۱۴۰۰'): 'both_1400',
 
 ('14011', 'ترم های 4001 - 4002 - 4011', 'مهر 1401', 'مهر ۴۰۱', 'پاییز ۱۴۰۱'): 'first_1401',
 ('بهار 1402', 'بهار ۱۴۰۲', 'بهار ۴۰۲', 'بهمن 1401', 'بهمن ۴۰۱', 'فرودین ۱۴۰۲'): 'second_1401',
 ('تابستان 1401',): 'summer_1401',
 ('مهر 1401 و بهمن 1402',): 'both_1401',

 ('1402 پاییز', 'مهر 1402', 'مهر 402', 'مهر ۱۴۰۲', 'مهر ۴۰۲', 'مهر۱۴۰۲', 'مهر۴۰۲', 'مهر 1402 - بهمن 1402', 'مهر ۱۴۰۲-بهمن ۱۴۰۲', 'بهمن۱۴۰۱_مهر۱۴۰۲', 'پاییز  ۱۴۰۲', 'پاییز 1402', 'پاییز 402', 'پاییز ۱۴۰۲', '۱۴۰۲ مهر', '۱۴۰۲-۱'): 'first_1402',
 ('4022', 'اسفند 1402', 'اسفند ۱۴۰۲', 'بهار 1403', 'بهار ۱۴۰۳', 'بهار ۴۰۳', 'بهار 403', 'بهتر ۴۰۳', 'بهمن ۱۰۴۲', 'بهمن ۴۰۲ ۴۰۳', 'بهمن۴۰۲-۳', 'بهمن 1402', '1402 بهمن', '1402بهمن', 'بهمن  1402', 'بهمن 1402-1403', 'بهمن 402', 'بهمن ۱۴۰۲', 'بهمن ۴۰۲', 'بهمن۱۴۰۲', 'ترم دو ۱۴۰۲_۱۴۰۳', 'زمستان ۱۴۰۲', 'زمستان ۴۰۲', 'زمستان 402', 'خرداد ۴۰۳', 'زمستان ۱۴۰۲ و بهار ۱۴۰۳', 'زمستان ۴۰۳', '۱۴۰۲٫۱۴۰۳', '۱۴۰۲_۱۴۰۳', '۴۰۲.۲'): 'second_1402',
 ('تابستان 1402',): 'summer_1402',
 ('1402', '۱۴۰۲', 'اول و دوم ۱۴۰۲', 'مهر 1402 و بهمن 1402', 'مهر و بهمن ۴۰۲', '۱۴۰۲ ۱۴۰۳'): 'both_1402',
 
 ('1403 مهر', '403', '4031', 'مهر 1403', 'مهر 403', 'مهر ۱۴٠۳', 'مهر ۱۴۰۳', 'مهر ١۴٠٣', 'مهر ۴۰۳', 'مهر1403', 'مهر403', 'مهر۱۴۰۳', 'مهر۴۰۳', 'مهر۴۰۳- بهمن۴۰۳', 'نیمه اول ۱۴۰۳', 'پائیز 1403', 'پاییز 1403', 'پاییز 403', 'پاییز ۱۴۰۳', 'پاییز ۴۰۳', 'پاییز۴۰۳'): 'first_1403',
 ('1403-2', '4032', '403_2', 'بهار ۱۴۰۴', 'بهار ۴۰۴','بهمن 1403', 'بهمن 403', 'بهمن ١٤٠٣', 'بهمن ۱۴٠۳', 'بهمن ۱۴۰۳', 'بهمن ۴۰۳', 'بهمن۱۴۰۳', 'بهمن ۴۰۲-۳', 'بهمن۴۰۳', 'ترم زمستان 1403', '۴۰۳-۲', '۴۰۳۲'): 'second_1403',
 ('تابستان 1403', 'تابستان 403', 'تابستان ۱۴۰۳', 'تابستان ۴۰۳'): 'summer_1403',
 ('۱۴۰۳', '۴۰۳', 'آبان و بهمن ۱۴۰۳', 'بهمن ۱۴۰۳ و مهر ۱۴۰۳', 'مهر و بهمن 1403', 'مهر و بهمن ۴۰۳', 'مهر ۱۴۰۳ و بهمن ۴۰۳-۴۰۴', 'مهر۱۴۰۳و بهمن ۱۴۰۳'): 'both_1403',

 ('مهر 1404', '۴۰۴'): 'first_1404',
 ('بهمن 1404', 'بهمن ۱۴۰۴', 'بهمن ۴۰۴'): 'second_1404',
 ('تابستان 1404', 'تابستان ۴٠۴'): 'summer_1404'
}

flat_mapping = {
    key: value
    for keys, value in temp_grouped.items()
    for key in keys
}

In [7]:
df['term'] = (
    df['term']
      .astype(str)
      .str.strip()
      .str.replace(r'([آ-یA-Za-z]+)([0-9]+)', r'\1 \2', regex=True)
      .map(flat_mapping)
)

### Professor Names Normalization

In [8]:
# Checking For Common Prefixes
prefixes = ['سید', 'دکتر', 'مهندس', 'آقای', 'آقا', 'خانم', 'خانوم', 'سرکار', 'سرکار خانم', 'سرکار خانوم', 'استاد', 'پروفسور', 'پرفسور', 'جناب', 'جناب آقای']

pc = {p: 0 for p in prefixes}
for n in df['professor_name_raw']:
    s = str(n).strip()
    for p in prefixes:
        if s.startswith(p):
            pc[p] += 1
            break

for p, c in sorted(pc.items(), key=lambda x: -x[1]):
    if c: print(f"{p}: {c}")

دکتر: 153
سید: 143
استاد: 88
خانم: 2
پروفسور: 2
مهندس: 1
آقای: 1
آقا: 1
پرفسور: 1


In [9]:
# Cheking For Combined Names & Store Them For Later
mask = df['professor_name_raw'].fillna('').str.contains(r'، | و |-', na=False)
combined = df[mask]
combined

,message_id,date,term,professor_id,professor_name_raw,department,course_name,coherence,knowledge,teaching,management,responsiveness,behavior,grading_status_raw,attendance_status_raw,comment_text,parse_error
1013,1063,2022-07-07 14:59:37,second_1400,NaN,استاد سیدین و ابوطالبی,مهندسی_مواد,پدیده های انتقال,8.0,8.0,8.0,8.0,6.0,8.0,منصفانه و هرچی خودت بگیری,حضور مهم است و تاثیر مستقیم دارد,استاد سیدین خوش نمره نیست امتحاناش هم سخته ولی...,NaN
1946,2045,2023-06-25 17:18:47,second_1401,NaN,اکبری - محمودوند,مکانیک,کارگاه اتومکانیک,7.0,2.0,5.0,5.0,7.0,5.0,منصفانه و هرچی خودت بگیری,حضور مهم است و تاثیر مستقیم دارد,کلاس‌ها دو صورت تئوری و عملی داره. اول تئوری گ...,NaN
1981,2080,2023-06-28 12:49:01,second_1401,NaN,معصومه احمدی-زهرا عابدینی,فیزیک,آز فیزیک ۲,10.0,10.0,10.0,10.0,10.0,10.0,دست باز و با ارفاق,حضور مهم است و تاثیر مستقیم دارد,هر دو استاد بسیار خوب و خوش برخورد هستند و تدر...,NaN
2025,2124,2023-06-30 18:22:12,second_1401,NaN,خانم احمدی و عابدینی,فیزیک,آز ۲,7.0,7.0,7.0,8.0,10.0,10.0,دست باز و با ارفاق,حضور مهم است و تاثیر مستقیم دارد,۸ نمره گزارش کار\n۸ نمره امتحان عملی\n۴ نمره ا...,NaN
2066,2165,2023-07-01 21:42:42,second_1401,NaN,دکتر زمانی-مهندس احمدی,فیزیک,آزمایشگاه فیزیک ۲,10.0,10.0,10.0,10.0,10.0,10.0,دست باز و با ارفاق,حضور مهم است و تاثیر مستقیم دارد,چیزی اضافه ایی نیست,NaN
2118,2217,2023-07-02 21:36:44,second_1401,NaN,نوروزبیگی-سعیدزاده,مهندسی_شیمی,آز حرارت,10.0,10.0,10.0,10.0,10.0,10.0,دست باز و با ارفاق,حضور مهم است و تاثیر مستقیم دارد,دکتر نوروزبیگی با شخصیت و مسلط به مباحث هستند،...,NaN
2445,2555,2023-07-15 18:18:08,second_1401,NaN,استاد عابدینی، داوودی و اکبری,مکانیک,روش های تولید,3.0,9.0,1.0,2.0,5.0,10.0,نمره خوبی نمیشه ازشون گرفت,حضور و غیاب نمی کند,فقط استاد عابدینی هم تدریس و هم نمره دهی نرمال...,NaN
2605,2725,2024-01-27 22:48:23,first_1402,NaN,استاد چوپانکاره و استاد میرشفیعی,برق,کارگاه برق,9.0,10.0,8.0,8.0,7.0,9.0,منصفانه و هرچی خودت بگیری,حضور مهم است و تاثیر مستقیم دارد,همه امتحان عملی هاشو حصور داشته باشید و قبلش ب...,NaN
2635,2756,2024-01-27 22:58:03,first_1402,NaN,مجید نوروزی - امید محمدیان,تربیت_بدنی,شنا,10.0,10.0,10.0,10.0,10.0,10.0,دست باز و با ارفاق,حضور مهم است و تاثیر مستقیم دارد,همه بچه هایی که با این دو استاد برداشتن توی یک...,NaN
2708,2830,2024-01-28 17:26:35,first_1402,NaN,مریم صالحی و حمید رضا کوهدار,مهندسی_مواد,خواص فیزیکی 2,10.0,10.0,10.0,10.0,10.0,10.0,دست باز و با ارفاق,حضور مهم نیست اما تاثیر مثبت دارد,جفت استادا عالی درس میدن و امتحان از جزوه خودش...,NaN


In [10]:
src_col = 'professor_name_raw'
out_col = 'prof_merged'

# INITIAL NORMALIZE
CHAR_MAP = {'ي':'ی','ى':'ی','ك':'ک','ؤ':'و','ے':'ی','أ':'ا','إ':'ا','ٱ':'ا','ـ':''}
ZW_RE = re.compile(r'[\u200c\u200d\uFEFF]')
PUNCT_RE = re.compile(r'[^\w\sآ-ی]')

def basic_normalize(s):
    if not isinstance(s, str):
        return ''
    s = unicodedata.normalize('NFKC', s.strip())
    s = ''.join(ch for ch in s if not unicodedata.combining(ch))
    for a, b in CHAR_MAP.items():
        s = s.replace(a, b)
    s = ZW_RE.sub('', s)
    s = PUNCT_RE.sub(' ', s)
    return re.sub(r'\s+', ' ', s).strip()

# Prefixes Removal
PREFIXES = ['سیده', 'سید', 'مهندس', 'استاد', 'خانم', 'پروفسور', 'پرفسور', 'آقا', 'آقای', 'اقای دکتز']
pat_start = re.compile(r'^\s*(?:' + '|'.join(map(re.escape, PREFIXES)) + r')(?=\s|[آ-ی]|$)\s*')

# Special Case
SPECIAL_CASES = ['ریاضی دو', 'زبان عمومی']
pat_special = re.compile(r'\b(?:' + '|'.join(map(re.escape, SPECIAL_CASES)) + r')\b')

# Special Token Fix
FIXES = [
    (re.compile(r'\bاقا(?=[آ-ی])'), 'آقا'),
    (re.compile(r'\bاشتیانی\b'), 'آشتیانی'),
    (re.compile(r'\bازاد\b'), 'آزاد'),
    (re.compile(r'\bاسمعیلی\b'), 'اسماعیلی'),
    (re.compile(r'\bاللهی\b'), 'الهی'),
    (re.compile(r'الله'), 'اله'),
    (re.compile(r'الله'), 'اله'),
]


def clean_name(s):
    s = basic_normalize(s)

    has_doc = bool(re.search(r'\bدکتر\b', s))
    s = re.sub(r'\bدکتر\b', ' ', s)

    s = pat_special.sub(' ', s)

    while True:
        new = pat_start.sub('', s)
        if new == s:
            break
        s = new

    for p, r in FIXES:
        s = p.sub(r, s)

    s = re.sub(r'\s+', ' ', s).strip()
    return s, has_doc

def format_rep(base, has_doc):
    return ('دکتر ' + base) if (has_doc and base) else base

In [11]:
DEPT_TH = {'فیزیک': 0.75, 'مکانیک': 0.75, 'مهندسی_مواد': 0.80, 'مهندسی_کامپیوتر': 0.71,
'برق': 0.76, 'مهندسی_عمران': 0.71, 'معارف': 0.79, 'مهندسی_شیمی': 0.80, 'ریاضی': 0.72, 'مهندسی_راه_آهن': 0.79,}

def name_similarity(a, b, dept):
    a = re.sub(r'\s+', ' ', a.strip())
    b = re.sub(r'\s+', ' ', b.strip())
    if not a or not b:
        return False

    a_tokens = a.split()
    b_tokens = b.split()
    num_a = len(a_tokens)
    num_b = len(b_tokens)

    threshold = DEPT_TH.get(dept, 0.75)

    if num_a == 1 and num_b == 1:
        threshold = 0.85

    is_single_vs_last = False
    if min(num_a, num_b) == 1 and max(num_a, num_b) > 1:
        short_token = a_tokens[0] if num_a == 1 else b_tokens[0]
        long_tokens = b_tokens if num_a == 1 else a_tokens
        if short_token == long_tokens[-1]:
            is_single_vs_last = True

    a_no_space = ''.join(a_tokens)
    b_no_space = ''.join(b_tokens)
    similarity = difflib.SequenceMatcher(None, a_no_space, b_no_space).ratio()

    if is_single_vs_last:
        return True

    return similarity >= threshold

In [12]:
def merge_names_by_department(df, src_col, out_col):
    df[out_col] = ''

    for department, sub in df.groupby('department'):
        key_to_rows = defaultdict(list)

        for idx, orig in sub[src_col].fillna('').items():
            base, has_doc = clean_name(orig)
            if base:
                key_to_rows[base].append((idx, has_doc))

        keys = list(key_to_rows)
        if not keys:
            continue

        G = nx.Graph()
        G.add_nodes_from(keys)

        for a, b in combinations(keys, 2):
            if name_similarity(a, b, department):
                G.add_edge(a, b)

        for cluster in nx.connected_components(G):
            rep_base = max(
                cluster,
                key=lambda k: (len(key_to_rows[k]), len(k.split()))
            )
            doc_present = any(
                has_doc for k in cluster for _, has_doc in key_to_rows[k]
            )
            rep = format_rep(rep_base, doc_present)

            for k in cluster:
                for idx, _ in key_to_rows[k]:
                    df.at[idx, out_col] = rep

    return df

In [13]:
MANUAL_MERGE = {
    'فیزیک': {
        'راه چمنی': 'ذبیح اله راه چمنی',
        'راهچمنی': 'ذبیح اله راه چمنی',
        'زهرا عابدینی امین آباد': 'زهرا عابدینی',
        'زهرا کی احمدی': 'زهرا کی احمدی'
    },
    'مکانیک': {
        'آیت الهی': 'مجیدرضا آیت الهی'
    },
    'مهندسی_مواد': {
        'رضوی عشیری': 'روح الهی عشیری'
    },
    'مهندسی_کامپیوتر': {
        'زهرا رشیدی': 'زهرا رشیدی'
    },
    'برق': {
        'رحیمی پور': 'رحیمی پور',
        'داوود عرب': 'داوود عرب خابوری',
        'عرب': 'داوود عرب خابوری',
        'حسین عرب': 'حسین عرب',
        'راحمی': 'راحمی نوش آبادی',
        'حاجی احمدی': 'محمد جواد حاجی احمدی'
    },
    'مهندسی_عمران': {
        'علی کاوه': 'علی کاوه',
        'علیزاده': 'حسین علیزاده',
        'حسین علیزاده': 'حسین علیزاده'
    },
    'معارف': {
        'رهبری': 'سعید رهبری شمیرانی',
        'سعید رهبری': 'سعید رهبری شمیرانی',
        'هوشمند': 'نرگس هوشمند همدانی',
        'نرگس هوشمند': 'نرگس هوشمند همدانی',
        'حسن محسنی': 'حسن محسنی',
        'شاه علی': 'احمدرضا شاه علی'
        
    },
    'مهندسی_شیمی': {
        'احمد رهبر': 'احمد رهبر کلیشمی',
        'رهبر': 'احمد رهبر کلیشمی'
    },
    'ریاضی': {
        'محمدهادی علاییان': 'محمد هادی علائیان',
        'محمدهادی علائیان': 'محمد هادی علائیان',
        'محمد هادی علائیان': 'محمد هادی علائیان',
        'جانی': 'اسداله آقاجانی'
    },
    'مهندسی_راه_آهن': {
        'حمید رضا احدی': 'حمیدرضا احدی',
        'حمیدرضا احدی': 'حمیدرضا احدی',
        'حمیدرضا حیدری': 'حمیدرضا حیدری',
        'حمیدرضا حیدری نوقابی': 'حمیدرضا حیدری'
    }
}


def apply_manual_merge(df, src_col, out_col):
    for dept, rules in MANUAL_MERGE.items():
        for src, dst in rules.items():
            src_norm, _ = clean_name(src)
            dst_norm, _ = clean_name(dst)

            if not src_norm or not dst_norm:
                continue

            mask = (
                (df['department'] == dept) &
                (df[src_col].apply(lambda x: clean_name(x)[0]) == src_norm)
            )

            df.loc[mask, out_col] = dst_norm

    return df


In [14]:
def print_clusters_by_department(df, src_col, out_col, dept):
    sub = df[df['department'] == dept]
    if sub.empty:
        print(f"No data for department: {dept}")
        return
    print(f"Department: {dept}")
    cluster_to_members = defaultdict(list)
    for _, row in sub.iterrows():
        base, _ = clean_name(row[src_col])
        if base:
            cluster_to_members[row[out_col]].append(base)
    for rep, members in sorted(cluster_to_members.items()):
        unique_members = sorted(set(members))
        print(f"{unique_members} : {rep}")

In [15]:
dfn = df[~mask].copy()
dfn = merge_names_by_department(dfn, src_col, out_col)
dfn = apply_manual_merge(dfn,src_col, out_col)

In [16]:
dfn['department'].unique()

array(['برق', 'مهندسی_کامپیوتر', 'مهندسی_مواد', 'مکانیک', 'فیزیک',
       'ریاضی', 'مهندسی_صنایع', 'مهندسی_شیمی', 'معارف', 'مهندسی_عمران',
       'تربیت_بدنی', 'مهندسی_راه_آهن', 'معماری_و_شهرسازی', 'شیمی',
       'طراحی_صنعتی', 'مهندسی_خودرو', 'مهندسی_پیشرفت', 'فناوری_های_نوین'],
      dtype=object)

In [17]:
print_clusters_by_department(dfn, src_col, out_col, 'ریاضی')

Department: ریاضی
['ابراهیم اردشیر لاریجانی', 'اردشیر لاریجانی'] : ابراهیم اردشیر لاریجانی
['احمد رضا فروغ', 'احمد فروغ', 'احمدرضا فروغ', 'فروغ'] : احمدرضا فروغ
['ادریس فیض آبادی'] : ادریس فیض آبادی
['آقاجانی', 'اسداله آقاجانی', 'جانی'] : اسداله آقاجانی
['افشین نمیرانیان'] : افشین نمیرانیان
['اکبر دهقان نژاد', 'دهقان نژاد'] : اکبر دهقان نژاد
['تورج نیک آزاد', 'نیک آزاد'] : تورج نیک آزاد
['تومانیان مگردیچ'] : تومانیان مگردیچ
['جلیل رشیدی نیا', 'جلیل رشیدینیا', 'رشیدی نیا'] : جلیل رشیدی نیا
['حسن خرازی', 'خرازی'] : حسن خرازی
['حسینی پور'] : حسینی پور
['جوادی', 'حاجی سید جوادی', 'حمید حاجی سید جوادی'] : حمید حاجی سید جوادی
['حمید مسگرانی', 'مسگرانی'] : حمید مسگرانی
['حمیدرضا نوش کاران', 'حمیدرضا نوشکاران', 'نوشکاران'] : حمیدرضا نوشکاران
['حنیف حیدری'] : حنیف حیدری
['الهام اله مرادی'] : دکتر الهام اله مرادی
['الهام فتاحی', 'فتاحی'] : دکتر الهام فتاحی
['امید نیکان', 'مجیدیان'] : دکتر امید نیکان
['جواد وحیدی'] : دکتر جواد وحیدی
['رضا احمدی'] : دکتر رضا احمدی
['رضا سعادتی', 'سعادتی'] : دکتر ر

In [18]:
dept_name = 'ریاضی'

original_max_rows = pd.get_option('display.max_rows')
original_max_columns = pd.get_option('display.max_columns')
original_width = pd.get_option('display.width')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

filtered = (
    dfn.loc[
        (dfn['department'] == dept_name) & (dfn[src_col] != dfn[out_col]),
        [src_col, out_col, 'department']
    ]
    .drop_duplicates()
)

display(filtered)

pd.set_option('display.max_rows', original_max_rows)
pd.set_option('display.max_columns', original_max_columns)
pd.set_option('display.width', original_width)

,professor_name_raw,prof_merged,department
41,الهام فتاحی,دکتر الهام فتاحی,ریاضی
42,رضا سعادتی,دکتر رضا سعادتی,ریاضی
43,احمد فروغ,احمدرضا فروغ,ریاضی
51,سمیه سعیدی نژاد,دکتر سمیه سعیدی نژاد,ریاضی
58,سمانه مشهدی,دکتر سمانه مشهدی,ریاضی
61,محمدهادی علائیان,محمد هادی علائیان,ریاضی
66,مهدی نجفی خواه,دکتر مهدی نجفی خواه,ریاضی
89,استاد زهره مستقیم,زهره مستقیم,ریاضی
100,مسعود هادیان دهکردی,دکتر مسعود هادیان,ریاضی
135,علی رضا مصلح تهرانی,علیرضا مصلح تهرانی,ریاضی
